In [96]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
import dash
from dash import dcc
from dash import html
import plotly.express as px
import plotly.graph_objects as go
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from bson import ObjectId

# GLOBAL VARIABLES
# config_Cat = {validVals:}

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "securePassword"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
# Query the database
query = {}
readExactMatch = False
findOne = False
df = pd.DataFrame.from_records(db.read(query, readExactMatch, findOne))

#Data cleanse
df.drop(columns=['_id'], inplace=True) 

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'assets/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# layout
app.layout = html.Div([
    html.Div(
    [
        # clickable logo
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height': '140px', 
                    'width': '140px', 
                    'position': 'relative', 
                    'z-index': '0'}
            ),
            href="https://www.snhu.edu"
        )
    ],
        style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center'}
    ),
    # unique identifier
    html.Div(
        html.P('Contract design by Allen White'),
        style={
            'display': 'flex', 
            'justify-content': 'space-between', 
            'align-items': 'left',
            'color': 'grey',  
            'font-size': '10px'  
        }
    ),
    # title
    html.Center(
        html.B(html.H1('SNHU CS-340 Dashboard'))
    ),
    html.Hr(),
    html.Div(
        # ALL FILTERS
        className='filters',
        style={'display': 'flex', 'justify-content': 'left', 'align-items': 'left'},
        children=[
            html.Div(
                className='animal-type-filter',
                style={
                    'flex': '1', 
                    'margin-right': '10px'},
                children=[
                    html.H3('Animal Type'),
                    dcc.Dropdown(
                        id='animal-type-dropdown',
                        options=[
                            {'label': 'All', 'value': 'All'},
                            {'label': 'Dog', 'value': 'Dog'},
                            {'label': 'Cat', 'value': 'Cat'}
                        ],
                        multi=False, # Only one at a time
                        value='All'  # Initialize to 'All'
                    )
                ]
            ),
            # RESCUE FILTER
            html.Div(
                id='rescue-type-filter',
                style={
                    'flex': '1', 
                    'margin-left': '10px', 
                    'display': 'none'
                },
                children=[
                    html.H3('Search and Rescue Type'),
                    dcc.Checklist(
                        id='rescue-type-checklist',
                        options=[
                            {'label': 'Mountain or Wilderness', 'value': 'Mountain or Wilderness'},
                            {'label': 'Water', 'value': 'Water'},
                            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'}
                        ],
                        value=[]  # Initialize value as empty list
                    )
                ]
            )
        ]
    ),
    html.Div(
    # RESET BUTTON
        html.Div(
            html.Button(
                'Reset', id='reset-button', n_clicks=0,
                style={
                    'float': 'left',
                    'position': 'relative',
                    'display': 'flex',
                    'align-items': 'center',
                    'height': '30%',
                    'margin-top': 'auto',
                    'margin-bottom': 'auto'
                },
            ),
            style={
                'height': '100px',
                'display': 'flex',
                'justify-content': 'left',
                'align-items': 'center'
            },
        ),
    ),
html.Hr(),

#INTERACTIVE TABLE
dash_table.DataTable(
    id='datatable-id',
    columns=[
        {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
    ],
    data=df.to_dict('records'),
    editable=False,
    filter_action="native",
    sort_action="native",
    sort_mode="multi",
    row_selectable="single",
    column_selectable=False,
    row_deletable=False,
    selected_columns=[],
    selected_rows=[],
    page_action="native",
    page_current=0,
    page_size=10
),
html.Br(),
html.Hr(),
html.Div(
    #CHART CONTAINER
    id='map-footer-container',
    style={
        'display': 'flex', 
        'justify-content': 'space-between'
    },
    children=[
        # MAPS
        html.Div(
            id='map-container',
            style={
                'position': 'relative', 
                'float':'left', 
                'height': '500px', 
                'width': '60%', 
                'border': '3px solid white'
            },
            children=[
                
                # LARGE MAP
                dl.Map(
                    id='map-id_large',
                    style={
                        'width': '100%', 
                        'height': '100%', 
                        'z-index': '1'
                    },
                    center=[30.75, -97.48],
                    zoom=10,
                    children=[dl.TileLayer(id="base-layer-id-large")]
                ),
                
                # MAP LABEL
                html.Div(
                    html.H3('No Animal Selected'),
                    id='map_label',
                    className='map-label', 
                    style={
                        'position': 'absolute',
                        'float': 'left',
                        'bottom': '10px',
                        'left': '10px',
                        'width': '300px',
                        'height': '200px',
                        'z-index': '4',
                        #'text-shadow': '2px 2px 4px rgba(0, 0, 0, 0.5)',
                        'background': 'rgba(255, 255, 255, 0.6)'#,
                        #'color': '#c9134b'
                    },
                ),
                
                # SMALL MAP
                dl.Map(
                    id='map-id_small',
                    style={
                        'position': 'absolute', 
                        'top': '10px', 
                        'right': '10px', 
                        'width': '200px',
                        'height': '200px', 
                        'border': '3px solid white', 
                        'z-index': '3'},
                    center=[30.75, -97.48],
                    zoom=7,
                    children=[dl.TileLayer(id="base-layer-id-small")]
                ),
            ]
        ),
        # PIE GRAPH
        html.Div(
            id='footer',
            style={
                'position': 'relative', 
                'float':'right', 
                'width': '40%', 
                'overflow': 'hidden'
            },
            children=[
                #html.H4('Results'),
                dcc.Graph(id="graph-id")
                #html.P("Breeds: ")
                ]
            )
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################


#CHECKLIST AND DROPDOWN
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-checklist', 'value'),
     Input('animal-type-dropdown', 'value')]
)
def update_data_table(rescue_type, animal_type_dropdown):
    if animal_type_dropdown == 'Dog':
        if rescue_type:
            print("rescue type: ", rescue_type)
            records = []
            
            #SEARCH FOR MTN OR WLDRNSS
            if 'Mountain or Wilderness' in rescue_type:
                query = {
                    "animal_type": animal_type_dropdown,
                    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male",
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                }
                #ADD TO rolling records
                records += db.read(query, readExactMatch, findOne)
                
            #SEARCH FOR WATER
            if 'Water' in rescue_type:
                query = {
                    "animal_type": animal_type_dropdown,
                    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                    "sex_upon_outcome": "Intact Female",
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                }
                #ADD TO rolling records
                records += db.read(query, readExactMatch, findOne) 
                
            #SEARCH DISASTER AND INVDL TRAINING
            if 'Disaster Rescue or Individual Tracking' in rescue_type:
                query = {
                    "animal_type": animal_type_dropdown,
                    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male",
                    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
                }
                #ADD TO rolling records
                records += db.read(query, readExactMatch, findOne)
                
            df = pd.DataFrame.from_records(records)
        else:
            df = pd.DataFrame.from_records(db.read({"animal_type": animal_type_dropdown}, readExactMatch, findOne))
   
    elif animal_type_dropdown == 'Cat':
        df = pd.DataFrame.from_records(db.read({"animal_type": animal_type_dropdown}, readExactMatch, findOne))
    else:
        df = pd.DataFrame.from_records(db.read({}, readExactMatch, findOne))

    #CATCH ERROR RETURN RECORDS
    if not df.empty and '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


# SHOW OPTIONS
@app.callback(
    Output('rescue-type-filter', 'style'),
    [Input('animal-type-dropdown', 'value')]
)
def show_rescueTypes(value):
    if value == 'Dog':
        return {'display': 'block'}  # show 
    else:
        return {'display': 'none'}  # hide 

# UPATE MAPS AND LABEL
@app.callback(
    [Output('map-id_large', "center"),
     Output('map-id_large', "zoom"),
     Output('map-id_large', "children"),
     Output('map-id_small', "center"),
     Output('map-id_small', "zoom"),
     Output('map-id_small', "children"),
     Output('map_label', "children")],
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "selected_rows")]
)
def update_maps(view_data, selected_rows):
    largeMapZoom = 10
    smallMapZoom = 7

    # Handle initial and edge case where no index is selected
    if view_data is None or selected_rows is None or len(selected_rows) == 0:
        selected_rows = [0]

    dff = pd.DataFrame.from_dict(view_data)

    map_large_children = [
        dl.TileLayer(id="base-layer-id-large"),
    ]
    map_small_children = [
        dl.TileLayer(id="base-layer-id-small"),
    ]
    map_label_children = []

    for i in selected_rows:
        lat = dff.iloc[i, dff.columns.get_loc('location_lat')]
        lon = dff.iloc[i, dff.columns.get_loc('location_long')]
        # UPDATE LARGE MAP
        map_large_children.append(
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(dff.iloc[i, dff.columns.get_loc('breed')]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[i, dff.columns.get_loc('name')])
                    ])
                ]
            )
        )
        # UPDATE SMALL MAP
        map_small_children.append(
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(dff.iloc[i, dff.columns.get_loc('breed')]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[i, dff.columns.get_loc('name')])
                    ])
                ]
            )
        )
        # UPDATE LABEL
        map_label_children.extend([
            html.H3("Selected Animal Details"),
            html.P("Type: " + dff.iloc[i, dff.columns.get_loc('animal_type')]),
            html.P("Breed: " + dff.iloc[i, dff.columns.get_loc('breed')]),
            html.P("Name: " + str(dff.iloc[i, dff.columns.get_loc('name')])),  # Convert name to string to catch edge cases
            html.P("Animal ID: " + dff.iloc[i, dff.columns.get_loc('animal_id')])
        ])

        # Update the center coordinates for each iteration
        map_large_center = [lat, lon]
        map_small_center = [lat, lon]

    return (
        map_large_center, largeMapZoom, map_large_children,
        map_small_center, smallMapZoom, map_small_children,
        map_label_children
    )
    
#PIE CHART
@app.callback(
    Output("graph-id", "figure"),
    [Input('datatable-id', 'data'),
    Input('animal-type-dropdown', 'value')]
)
def generate_chart(data, animals):
    df = pd.DataFrame.from_records(data)
    breed_counts = df['breed'].value_counts()
    animal_type_counts = df['animal_type'].value_counts()

    if animals == 'Cat' or animals == 'Dog':
        fig = go.Figure(data=[go.Pie(labels=breed_counts.index, values=breed_counts.values)])
    else:
        fig = go.Figure(data=[go.Pie(labels=animal_type_counts.index, values=animal_type_counts.values)])
    fig.update_traces(marker=dict(line=dict(width=1)))
    fig.update_layout(showlegend=True)
    
    
    return fig
    
# RESET BUTTON
@app.callback(
    Output('rescue-type-checklist', 'value'),
    Output('animal-type-dropdown', 'value'),
    [Input('reset-button', 'n_clicks')]
)
def reset_checklist_and_dropdown(n_clicks):
    if n_clicks > 0:
        return [], None
    else:
        raise dash.exceptions.PreventUpdate

    
if __name__ == '__main__':
    app.run_server(debug=False)
    app.config.suppress_callback_exceptions = True

10022
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', ''],
      dtype='object')
Dash is running on http://127.0.0.1:19709/

Dash app running on http://127.0.0.1:19709/
rescue type:  ['Water']
rescue type:  ['Water', 'Disaster Rescue or Individual Tracking']
rescue type:  ['Disaster Rescue or Individual Tracking']
rescue type:  ['Disaster Rescue or Individual Tracking', 'Mountain or Wilderness']
rescue type:  ['Disaster Rescue or Individual Tracking', 'Mountain or Wilderness', 'Water']
